#TWristAR_load_dataset.ipynb
Loads the dataset from the Zenodo repository and returns train and test X/y numpy arrays.  This file is an updated version of e4_get_xys and e4_load_data.

The basic flow is:
* Download and unzip the dataset if not already present
* Convert each session into IR1 format - a datetime indexed pandas dataframe with columns for each channel plus the label and subject number.
* Convert IR1 into IR2 - a set of three numpy arrays containing sliding window samples
   * X = (samples, time steps per sample, channels)  
   * y =  (samples, label) # activity classification  
   * s =  (samples, subject) # subject number
* Concatenate the individual IR2 arrays into a single IR3
* Separate the IR3 arrays into the final returned train/validate/test arrays using the subject dictionary

TWRistAR is small and easily downloadable so there is no option to used saved Intermediate Representations here as there is in some of the loaders for larger datasets.

Set interactive to true to run the Jupyter Notebook version.  Note most of the calls are setup to test the functions, not process the entire dataset, to do that set interactive to false and run all so that main executes.

This video describes the code https://mediaflo.txstate.edu/Watch/e4_data_processing. (updates have been made since this was made)


Acknowledgement to and a good example of the WISDM format being pre-processed is https://towardsdatascience.com/human-activity-recognition-har-tutorial-with-keras-and-core-ml-part-1-8c05e365dfa0  by Nils Ackermann.  


<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

[Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), Texas State University, [IMICS Lab](https://imics.wp.txstate.edu/)  
TODO:
* IR2/3 label dtypes are strings - not sure if strings are the best answer or should use number along with "decoder" dictionary.
* log_info needs to be updated to dictionary format so we can read things like the channel names automatically.
* Time is off by 6 hrs due to time zone issues - adjusted in Excel/csv but would be good to show it in the correct time zone.
* Need to incorporate session numbers or just use the alternate .csv files where validation was 'fake' subs 11 and 22 which were just a few of the sessions from subjects 1 and 2.  This was done in the Semi-Supervised version of the loader for WISHWell but not integrated back into this version.


In [ ]:
import os
import shutil #https://docs.python.org/3/library/shutil.html
from shutil import unpack_archive # to unzip
#from shutil import make_archive # to create zip for storage
#import requests #for downloading zip file
#from scipy import io #for loadmat, matlab conversion
import time
import pandas as pd
import numpy as np
from numpy import savetxt
from tabulate import tabulate # for verbose tables, showing data
from tensorflow.keras.utils import to_categorical # for one-hot encoding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#import matplotlib.pyplot as plt # for plotting - pandas uses matplotlib
from time import gmtime, strftime, localtime #for displaying Linux UTC timestamps in hh:mm:ss
from datetime import datetime, date
import urllib.request # to get files from web w/o !wget

In [ ]:
my_dir = '.' # replace with absolute path if desired
dataset_dir = my_dir # TWristAR zip file contains TWristAR directory
working_dir = os.path.join(my_dir,'TWristAR_temp') # temp dir for processing

# if not os.path.exists(working_dir):
#     os.makedirs(working_dir)
interactive = True # for exploring data and functions interactively
verbose = True

In [ ]:
interactive = False # don't run if interactive, automatically runs for .py version
verbose = False # to limit the called functions output

In [ ]:
def get_TWristAR():
    """checks for local zipfile, if none downloads from zenodo repository
    after download will unzip the dataset into TWristAR directory.
    Assumes a global my_dir has been defined (default is my_dir = ".")
    :return: nothing"""
    zip_ffname = os.path.join(my_dir,'TWristAR.zip')
    if (os.path.exists(zip_ffname)):
        if verbose:
            print ("Local TWristAR.zip found, skipping download")
    else:
        print("Downloading TWristAR from Zenodo")
        urllib.request.urlretrieve("https://zenodo.org/record/5911808/files/TWristAR.zip", filename="TWristAR.zip")
    if (os.path.isdir(os.path.join(dataset_dir,'TWristAR'))):
        if verbose:
            print("Found existing TWristAR directory, skipping unzip")
        return
    else:
        print("Unzipping TWristAR file in", dataset_dir, "directory")
        if (os.path.exists(zip_ffname)):
            shutil.unpack_archive(zip_ffname,dataset_dir,'zip')
        else:
            print("Error: ", zip_ffname, " not found, exiting")
            return
if interactive:
    get_TWristAR()

Unzipping TWristAR file in . directory


In [ ]:
def unzip_e4_file(zip_ffname, working_dir):
    """checks for local copy, if none unzips the e4 zipfile in working_dir
    Note:  the files themselves do not contain subject info and there are
    multiple files e.g. ACC.csv, BVP,csv etc, in each zipfile.
    It is very important to further process the files with <fname>_labels.csv
    :param zip_ffname: the path and filename of the zip file
    :param working_dir: local (colab) directory where csv files will be placed
    :return: nothing"""
    if not os.path.exists(working_dir):
        print("Error working directory", working_dir, "not found, unzip_e4_file exiting")
        return
    if (os.path.exists(zip_ffname)):
        if verbose:
            print("Unzipping",zip_ffname, "in", working_dir)
        shutil.unpack_archive(zip_ffname,working_dir,'zip')
    else:
        print("Error: ", zip_ffname, " not found, exiting")
        return
if interactive:
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)
    zip_ffname = os.path.join(my_dir,'TWristAR','sub1/1574621345_A01F11.zip')
    unzip_e4_file(zip_ffname, working_dir)

Unzipping ./TWristAR/sub1/1574621345_A01F11.zip in ./TWristAR_temp


In [ ]:
def df_from_e4_csv (ffname,col_labels):
    """"reads e4 ACC, BVP, EDA, or TEMP(erature) csv files, uses start time and
    sample rate to create time indexed pandas dataframe with columns.  
    Note the other e4 files have different format and must be read seperately. 
    :param ffname:  full filename e.g./content/temp/ACC.csv
    :col_labels: list of colums in csv - varies by type ['accel_x','accel_y...]
    :returns df: time indexed dataframe"""

    df = pd.read_csv(ffname, header=None)
    start_time = df.iloc[0,0].astype('int64') # first line in e4 csv
    sample_freq = df.iloc[1,0].astype('int64') # second line in e4 csv
    df = df.drop(df.index[[0,1]]) # drop 1st two rows, index is now off by 2
    # Make the index datetime first so code can be used for other data types
    # Having the index as datetime is required for pandas resampling
    # The start_time from the e4 csv file is forced to int64 which represents the
    # number of nanoseconds since January 1, 1970, 00:00:00 (UTC)
    # This is tricky - if float representation the join function may not work
    # properly later since the indexes must match exactly.
    # UTC_time is computed for each row, then made into required datetime format
    # that pandas will accept as an index
    df['UTC_time'] = (df.index-2)/sample_freq + start_time
    end_time = df['UTC_time'].iloc[-1]
    if verbose:
        print(ffname, "Sample frequency = ", sample_freq, " Hz")
        #show time in day month format, assumes same timezone
        print("File start time = ", strftime("%a, %d %b %Y %H:%M:%S", localtime(start_time)))  
        print("File end time   = ",strftime("%a, %d %b %Y %H:%M:%S", localtime(end_time)))
    #df = df.astype({'UTC_time': 'int64'}) # change future index from float64 to int64
    # this causes issues when trying to synch across sensors, if float then
    # some amount of error will be introduced.   May need to move earlier.
    df['datetime'] = pd.to_datetime(df['UTC_time'], unit='s')
    df.set_index('datetime',inplace=True)
    df = df.drop('UTC_time', axis=1)
    df.columns = col_labels
    return df
if interactive:
    # Note: IBI.csv is the inter-beat interval, a calculated value with a 
    # different format.  HR.csv is also calculated from BVP but format is same.
    ffname = working_dir + '/ACC.csv'
    col_labels = ['accel_x', 'accel_y', 'accel_z']
    ir1_acc_df = df_from_e4_csv(ffname, col_labels)
    print("ACC dataframe shape", ir1_acc_df.shape)
    display(ir1_acc_df.head())

    ffname = working_dir + '/BVP.csv'
    col_labels = ['bvp']
    ir1_bvp_df = df_from_e4_csv(ffname, col_labels)
    print("BVP dataframe shape", ir1_bvp_df.shape)
    display(ir1_bvp_df.head())

    ffname = working_dir + '/EDA.csv'
    col_labels = ['eda']
    ir1_eda_df = df_from_e4_csv(ffname, col_labels)
    print("EDA dataframe shape", ir1_eda_df.shape)
    display(ir1_eda_df.head())

    ffname = working_dir + '/TEMP.csv'
    col_labels = ['p_temp']
    ir1_temp_df = df_from_e4_csv(ffname, col_labels)
    print("Temp dataframe shape", ir1_temp_df.shape)
    display(ir1_temp_df.head())

./TWristAR_temp/ACC.csv Sample frequency =  32  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
ACC dataframe shape (17502, 3)


,accel_x,accel_y,accel_z
datetime,,,
2019-11-24 18:49:05.000000000,-2.0,-26.0,57.0
2019-11-24 18:49:05.031249920,-3.0,-26.0,58.0
2019-11-24 18:49:05.062500096,-3.0,-26.0,58.0
2019-11-24 18:49:05.093750016,-4.0,-26.0,58.0
2019-11-24 18:49:05.124999936,-4.0,-26.0,58.0


./TWristAR_temp/BVP.csv Sample frequency =  64  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
BVP dataframe shape (35002, 1)


,bvp
datetime,
2019-11-24 18:49:05.000000000,-0.0
2019-11-24 18:49:05.015624960,-0.0
2019-11-24 18:49:05.031249920,-0.0
2019-11-24 18:49:05.046874880,-0.0
2019-11-24 18:49:05.062500096,-0.0


./TWristAR_temp/EDA.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
EDA dataframe shape (2184, 1)


,eda
datetime,
2019-11-24 18:49:05.000000000,0.000000
2019-11-24 18:49:05.249999872,0.840149
2019-11-24 18:49:05.500000000,1.187489
2019-11-24 18:49:05.750000128,1.625562
2019-11-24 18:49:06.000000000,2.103368


./TWristAR_temp/TEMP.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
Temp dataframe shape (2184, 1)


,p_temp
datetime,
2019-11-24 18:49:05.000000000,31.83
2019-11-24 18:49:05.249999872,31.83
2019-11-24 18:49:05.500000000,31.83
2019-11-24 18:49:05.750000128,31.83
2019-11-24 18:49:06.000000000,31.83


In [ ]:
def process_e4_accel(df):
    """converts component accel into g and adds accel_ttl column
    per info.txt range is [-2g, 2g] and unit in this file is 1/64g.
    """
    df['accel_x'] = df['accel_x']/64
    df['accel_y'] = df['accel_y']/64
    df['accel_z'] = df['accel_z']/64
    df_sqd = df.pow(2)[['accel_x', 'accel_y', 'accel_z']] #square each accel
    df_sum = df_sqd.sum(axis=1) #add sum of squares, new 1 col df
    df.loc[:,'accel_ttl'] = df_sum.pow(0.5)-1  # sqrt and remove 1g due to gravity
    del df_sqd, df_sum
    return df
if interactive:
    ir1_acc_df = process_e4_accel(ir1_acc_df)
    display(ir1_acc_df.head())

,accel_x,accel_y,accel_z,accel_ttl
datetime,,,,
2019-11-24 18:49:05.000000000,-0.031250,-0.40625,0.890625,-0.020598
2019-11-24 18:49:05.031249920,-0.046875,-0.40625,0.906250,-0.005754
2019-11-24 18:49:05.062500096,-0.046875,-0.40625,0.906250,-0.005754
2019-11-24 18:49:05.093750016,-0.062500,-0.40625,0.906250,-0.004895
2019-11-24 18:49:05.124999936,-0.062500,-0.40625,0.906250,-0.004895


In [ ]:
def get_ir1_from_e4_dir(working_directory):
    """processes the four e4 sensor files into a single dataframe that
    is datetime indexed. Labeled columns are channels"""
    # Note: IBI.csv is the inter-beat interval, a calculated value with a 
    # different format.  HR.csv is also calculated from BVP but format is same.
    ffname = working_dir + '/ACC.csv'
    col_labels = ['accel_x', 'accel_y', 'accel_z']
    ir1_acc_df = df_from_e4_csv(ffname, col_labels)
    ir1_acc_df = process_e4_accel(ir1_acc_df)

    ffname = working_dir + '/BVP.csv'
    col_labels = ['bvp']
    ir1_bvp_df = df_from_e4_csv(ffname, col_labels)

    ffname = working_dir + '/EDA.csv'
    col_labels = ['eda']
    ir1_eda_df = df_from_e4_csv(ffname, col_labels)

    ffname = working_dir + '/TEMP.csv'
    col_labels = ['p_temp']
    ir1_ptemp_df = df_from_e4_csv(ffname, col_labels)

    ir1_df = ir1_acc_df.join(ir1_bvp_df, how="inner") # this drops bvp to 32Hz
    ir1_df = ir1_df.join(ir1_eda_df, how="outer") # stays at 32Hz, eda fill NaN
    ir1_df = ir1_df.join(ir1_ptemp_df, how="outer") # stays at 32Hz, p_temp fill NaN
    ir1_df = ir1_df.interpolate() # default is linear interpolation
    ir1_df = ir1_df.astype('float32') # no need for 64 precision with these sensors
    if verbose:
        print("IR1 full dataframe shape",ir1_df.shape)
        #print(ir1_df.head(10))
    return ir1_df
if interactive:
    ir1_df = get_ir1_from_e4_dir(working_dir)
    display(ir1_df.head(10))

./TWristAR_temp/ACC.csv Sample frequency =  32  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
./TWristAR_temp/BVP.csv Sample frequency =  64  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
./TWristAR_temp/EDA.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
./TWristAR_temp/TEMP.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
IR1 full dataframe shape (17501, 7)


,accel_x,accel_y,accel_z,accel_ttl,bvp,eda,p_temp
datetime,,,,,,,
2019-11-24 18:49:05.000000000,-0.031250,-0.40625,0.890625,-0.020598,-0.00,0.000000,31.83
2019-11-24 18:49:05.031249920,-0.046875,-0.40625,0.906250,-0.005754,-0.00,0.105019,31.83
2019-11-24 18:49:05.062500096,-0.046875,-0.40625,0.906250,-0.005754,-0.00,0.210037,31.83
2019-11-24 18:49:05.093750016,-0.062500,-0.40625,0.906250,-0.004895,-0.00,0.315056,31.83
2019-11-24 18:49:05.124999936,-0.062500,-0.40625,0.906250,-0.004895,-0.00,0.420074,31.83
2019-11-24 18:49:05.156250112,-0.062500,-0.40625,0.906250,-0.004895,0.00,0.525093,31.83
2019-11-24 18:49:05.187500032,-0.046875,-0.40625,0.906250,-0.005754,0.01,0.630112,31.83
2019-11-24 18:49:05.218749952,-0.046875,-0.40625,0.906250,-0.005754,-0.03,0.735130,31.83
2019-11-24 18:49:05.249999872,-0.031250,-0.40625,0.890625,-0.020598,-0.05,0.840149,31.83


In [ ]:
def show_tag_time(tag_ffname):
    """utility prints time marks from tags.csv to help with video sync 
    and labeling.   When this is run in colab it seems to be GMT regardless
    of timezone settings."
    :param tag_ffname: file to be processed e.g. /content/temp/tags.csv'
    :return: nothing"""
    df_temp = pd.read_csv(tag_ffname, header=None)
    df_temp.columns = ['UTC_time']
    print ("    UTC_time          Local Time")
    for index, row in df_temp.iterrows():
        print(index, row['UTC_time'],
            strftime("%a, %d %b %Y %H:%M:%S", localtime(row['UTC_time'])))
# https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
# link to string formats for date and time
if interactive:
    print('Tag info (button presses) from tags.csv')
    tag_ffname = working_dir + '/tags.csv'
    show_tag_time(tag_ffname)

Tag info (button presses) from tags.csv
    UTC_time          Local Time
0 1574621375.17 Sun, 24 Nov 2019 18:49:35
1 1574621774.22 Sun, 24 Nov 2019 18:56:14


In [ ]:
def label_df_from_csv (df, labels_ffname):
    """adds activity label and subject number to the dataframe based on the
    contents of a .csv file containing time and label info.
    Example csv format (see e4_time_sync.xlsx to help build csv from video)
        start,finish,label,sub
        2019:11:24 18:49:51,2019:11:24 18:50:18,Upstairs,1
        2019:11:24 18:50:18,2019:11:24 18:50:45,Downstairs,1
    :param df : time indexed dataframe from df_from_e4_csv method
    :labels_ffname : csv file with metadata
    :return : a dataframe with label and subject columns added"""
    df_labels = pd.read_csv(labels_ffname)
    df_labels['start'] =  pd.to_datetime(df_labels['start'], format='%Y:%m:%d %H:%M:%S')
    df_labels['finish'] =  pd.to_datetime(df_labels['finish'], format='%Y:%m:%d %H:%M:%S')
    # quick check to make sure all subjects are the same - only 1 sub per csv
    if (not (df_labels['sub'].eq(df_labels['sub'].iloc[0]).all())):
        print('Warning: Multiple subjects detected in csv, unusual for e4 data.')
    df['label']='Undefined' # add column with safe value for labels
    df['sub'] = np.NaN
    for index, row in df_labels.iterrows():
        #print(row['start'], row['finish'],row['label'])
        df.loc[row['start']:row['finish'],'label'] = row['label']
        df.loc[row['start']:row['finish'],'sub'] = row['sub']
    return df
if interactive:
    labels_ffname = os.path.splitext(zip_ffname)[0] + '_labels.csv'
    print("Adding label and sub info from ",labels_ffname)
    ir1_df = label_df_from_csv(ir1_df, labels_ffname)
    display(ir1_df[5000:5005]) # head is meaningless since start is undefined
    #ir1_df['label'].value_counts()
    print ("Label Counts - # samples before sliding window")
    print (ir1_df['label'].value_counts())

Adding label and sub info from  ./TWristAR/sub1/1574621345_A01F11_labels.csv


,accel_x,accel_y,accel_z,accel_ttl,bvp,eda,p_temp,label,sub
datetime,,,,,,,,,
2019-11-24 18:51:41.249999872,-0.250000,-0.906250,0.250000,-0.027226,-19.639999,12.577293,31.870001,Upstairs,1.0
2019-11-24 18:51:41.281250048,-0.218750,-0.875000,0.250000,-0.064064,-11.870000,12.577453,31.870001,Upstairs,1.0
2019-11-24 18:51:41.312499968,-0.234375,-0.859375,0.265625,-0.070477,-3.500000,12.577613,31.870001,Upstairs,1.0
2019-11-24 18:51:41.343749888,-0.265625,-0.890625,0.265625,-0.033395,5.910000,12.577773,31.870001,Upstairs,1.0
2019-11-24 18:51:41.375000064,-0.296875,-0.937500,0.281250,0.022811,12.870000,12.577933,31.870001,Upstairs,1.0


Label Counts - # samples before sliding window
Upstairs      6208
Downstairs    5889
Undefined     5404
Name: label, dtype: int64


In [ ]:
def get_ir2_from_ir1(df, time_steps, stride):
    """slice the IR1 dataframe into sliding window segments of
    time_steps length and return X, y, sub ndarrays.
    If stride = time_steps there is no overlap of the sliding window.
    This version does not use append, better for RAM
    df: pandas datetime indexed dataframe columns - channel(s), label, sub
    time_steps: number of samples in window, will discard a partial final window
    stride:  how far to move window, no overlap if equal to time_steps.
    """    
    # this was copied from SHL with improved memory capabilities
    # the channel list is in dataframe but not in the numpy arrays
    channel_list = list(df.columns)
    channel_list.remove('label') # need to make sure this is defined for IR1
    channel_list.remove('sub') # ditto - should probably add a check
    if verbose:
        print('Channels in X:',channel_list)
    X = df[channel_list].to_numpy(dtype = 'float32')
    #y = df['label'].to_numpy(dtype = 'int8') # doesn't work for strings
    y = df['label'].to_numpy(dtype='<U10')
    sub = df['sub'].to_numpy(dtype = 'int8')
    if verbose:
        print('X,y,sub array shapes before sliding window', X.shape, y.shape, sub.shape)
    #https://numpy.org/devdocs/reference/generated/numpy.lib.stride_tricks.sliding_window_view.html
    shapex = (time_steps,X.shape[1]) # samples (rows to include) and n-dim of original (all channels)
    shapey = (time_steps,) # samples (rows to include) and only one column
    shapesub = (time_steps,) # samples (rows to include) and only one column
    X = np.lib.stride_tricks.sliding_window_view(X, shapex)[::stride, :]
    X = X[:,0,:,:] # I admit I don't understand why this dimension appears...
    y = np.lib.stride_tricks.sliding_window_view(y, shapey)[::stride, :]
    sub = np.lib.stride_tricks.sliding_window_view(sub, shapesub)[::stride, :]
    if verbose:
        print('X,y,sub array shapes after sliding window', X.shape, y.shape, sub.shape)
    return X, y, sub, channel_list
if interactive:
    my_X, my_y, my_sub, all_channel_list = get_ir2_from_ir1(ir1_df, 96, 32)
    headers = ("array","shape", "object type", "data type")
    mydata = [("my_X:", my_X.shape, type(my_X), my_X.dtype),
            ("my_y:", my_y.shape ,type(my_y), my_y.dtype),
            ("my_sub:", my_sub.shape, type(my_sub), my_sub.dtype)]
    print("IR2 array info")
    print(tabulate(mydata, headers=headers))
    print("Returned all_channel_list", all_channel_list)

Channels in X: ['accel_x', 'accel_y', 'accel_z', 'accel_ttl', 'bvp', 'eda', 'p_temp']
X,y,sub array shapes before sliding window (17501, 7) (17501,) (17501,)
X,y,sub array shapes after sliding window (544, 96, 7) (544, 96) (544, 96)
IR2 array info
array    shape         object type              data type
-------  ------------  -----------------------  -----------
my_X:    (544, 96, 7)  <class 'numpy.ndarray'>  float32
my_y:    (544, 96)     <class 'numpy.ndarray'>  <U10
my_sub:  (544, 96)     <class 'numpy.ndarray'>  int8
Returned all_channel_list ['accel_x', 'accel_y', 'accel_z', 'accel_ttl', 'bvp', 'eda', 'p_temp']


In [ ]:
def clean_ir2(X, y, sub):
    """removes sliding windows containing NaN, multiple labels, or multiple
    subject numbers.  Collapses y, sub to column arrays.
    Returns cleaned versions of X, y, sub ndarrays"""
    # Copied directly from SHL.  Yay!
    # Check for NaN
    nans = np.argwhere(np.isnan(X))
    num_nans = np.unique(nans[:,0]) #[:,0] just 1st column index of rows w/ NaN
    if verbose:
        print(num_nans.shape[0], "NaN entries found, removing")
    idx = ~np.isnan(X).any(axis=2).any(axis=1)
    # this warrants some explanation!
    # any(axis=1) and 2 collapses channels and samples
    # good axis explanation https://www.sharpsightlabs.com/blog/numpy-axes-explained/
    # the ~ negates so NaN location are now False in the idx which is then
    # used to filter out the bad windows below
    X = X[idx]
    y = y[idx]
    sub = sub[idx]
    # repeat and confirm NaNs have been removed
    nans = np.argwhere(np.isnan(X))
    num_nans = np.unique(nans[:,0]) #[:,0] accesses just 1st column
    if (nans.size!=0):
        print("WARNING! Cleaned output arrays still contain NaN entries")
        print("execute print(X[99]) # to view single sample")
    # Now get rid of segments with multiple labels
    # Not happy with this code, must be a better way but it seems to work...
    idx = []
    for i in range(y.shape[0]):
        if np.all(y[i] == y[i][0]):
            idx.append(True)
            
        else:
            idx.append(False)
            #print('Discarding Row:', i)
    X = X[idx]
    y = y[idx]
    sub = sub[idx]
    # TODO check for multiple subjects in window
    y = y[:,0] # collapse columns
    y = y[np.newaxis].T  # convert to single column array
    sub = sub[:,0] # repeat for sub array
    sub = sub[np.newaxis].T
    return X, y, sub
if interactive:
    my_X, my_y, my_sub = clean_ir2(my_X, my_y, my_sub)
    print('IR2 shapes after cleaning', my_X.shape, my_y.shape, my_sub.shape)
    headers = ("array","shape", "object type", "data type")
    mydata = [("my_X:", my_X.shape, type(my_X), my_X.dtype),
            ("my_y:", my_y.shape ,type(my_y), my_y.dtype),
            ("my_sub:", my_sub.shape, type(my_sub), my_sub.dtype)]
    print("IR2 array info")
    print(tabulate(mydata, headers=headers))

0 NaN entries found, removing
IR2 shapes after cleaning (513, 96, 7) (513, 1) (513, 1)
IR2 array info
array    shape         object type              data type
-------  ------------  -----------------------  -----------
my_X:    (513, 96, 7)  <class 'numpy.ndarray'>  float32
my_y:    (513, 1)      <class 'numpy.ndarray'>  <U10
my_sub:  (513, 1)      <class 'numpy.ndarray'>  int8


In [ ]:
def drop_label_ir2_ir3(X, y, sub, label_to_drop):
    """removes windows with label = label_to_drop
    Returns updated version of X, y, sub"""
    # Also copied directly from SHL - double Yay!
    idx = []
    for i in range(y.shape[0]):
        if (y[i] == label_to_drop):
            idx.append(False)
        else:
            idx.append(True)
            #print('Discarding Row:', i)
    X = X[idx]
    y = y[idx]
    sub = sub[idx]
    return X, y, sub
if interactive:
    print("Label counts before drop")
    unique, counts = np.unique(my_y, return_counts=True)
    print (np.asarray((unique, counts)).T)
    print('X, y, sub array shapes before label drop', my_X.shape, my_y.shape, my_sub.shape)
    my_X, my_y, my_sub = drop_label_ir2_ir3(my_X, my_y, my_sub, 'Undefined')
    print("Label counts after drop")
    unique, counts = np.unique(my_y, return_counts=True)
    print (np.asarray((unique, counts)).T)
    print('IR2 shapes after label drop', my_X.shape, my_y.shape, my_sub.shape)
    headers = ("array","shape", "object type", "data type")
    mydata = [("my_X:", my_X.shape, type(my_X), my_X.dtype),
            ("my_y:", my_y.shape ,type(my_y), my_y.dtype),
            ("my_sub:", my_sub.shape, type(my_sub), my_sub.dtype)]
    print("IR2 array info after label drop")
    print(tabulate(mydata, headers=headers))

Label counts before drop
[['Downstairs' '170']
 ['Undefined' '163']
 ['Upstairs' '180']]
X, y, sub array shapes before label drop (513, 96, 7) (513, 1) (513, 1)
Label counts after drop
[['Downstairs' '170']
 ['Upstairs' '180']]
IR2 shapes after label drop (350, 96, 7) (350, 1) (350, 1)
IR2 array info after label drop
array    shape         object type              data type
-------  ------------  -----------------------  -----------
my_X:    (350, 96, 7)  <class 'numpy.ndarray'>  float32
my_y:    (350, 1)      <class 'numpy.ndarray'>  <U10
my_sub:  (350, 1)      <class 'numpy.ndarray'>  int8


In [ ]:
def get_ir3(
    working_dir = os.path.join(my_dir,'TWristAR_temp'), # dir will be created
    zip_flist = ['sub1/1574621345_A01F11.zip',
                'sub1/1574622389_A01F11.zip',
                'sub1/1574624998_A01F11.zip',
                'sub2/1633107019_A01F11.zip',
                'sub2/1633108344_A01F11.zip',
                'sub2/1633109744_A01F11.zip',
                'sub3/1633704587_A01F11.zip',
                'sub3/1633705664_A01F11.zip',
                'sub3/1633711821_A01F11.zip'],
    time_steps = 96,
    step = 32 #if equal to time_steps there will be no overlap of sliding window
    ):
    """This is the main function to create the three ndarrays X, y, sub from
    the dataset zip file.  It is an updated version of get_xys.  
    1) Downloads TWristAR from Zenodo repository,
    2) Processes each e4 zip file and associated label csv file into X (data),
     y (labels), and sub (subject number) IR2 ndarrays.
    3) Stackes the IR2 arrays into IR3 (X, y, sub) arrays
    Note:  TWristAR is purposely small, but for other datasets this can take
    a long time and it may be best to save these numpy arrays locally.
    """
    get_TWristAR()
    # create blank ndarrays to append to
    ir3_X = np.zeros(shape=(1,time_steps,7), dtype = 'float32')
    ir3_y = np.full(shape=(1,1), fill_value='n/a',dtype='<U10') # unicode 10 char
    ir3_sub = np.zeros(shape=(1,1),dtype=int) # one subject number per entry
    for i in zip_flist:
        zip_ffname = os.path.join(my_dir,'TWristAR',i)
        if verbose:
            print('Processing ', zip_ffname)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)
        unzip_e4_file(zip_ffname, working_dir)
        ir1_df = get_ir1_from_e4_dir(working_dir)
        if verbose:
            print('Tag info (button presses) from tags.csv')
            tag_ffname = working_dir + '/tags.csv'
            show_tag_time(tag_ffname)
        # Generate associated csv filename, forces the long numbered filenames to match
        labels_ffname = os.path.splitext(zip_ffname)[0] + '_labels.csv'
        ir1_df = label_df_from_csv (ir1_df, labels_ffname)
        ir1_df['label'].value_counts()
        if verbose:
            print ("Label Counts - # samples before sliding window\n",ir1_df['label'].value_counts())
        ir2_X, ir2_y, ir2_sub, channel_list = get_ir2_from_ir1(ir1_df, time_steps, step)
        ir2_X, ir2_y, ir2_sub = clean_ir2(ir2_X, ir2_y, ir2_sub)
        ir2_X, ir2_y, ir2_sub = drop_label_ir2_ir3(ir2_X, ir2_y, ir2_sub, 'Undefined')
        ir3_X = np.vstack([ir3_X, ir2_X])
        ir3_y = np.vstack([ir3_y, ir2_y])
        ir3_sub = np.vstack([ir3_sub, ir2_sub])
        shutil.rmtree(working_dir)

    #delete first row placeholders
    X = np.delete(ir3_X, (0), axis=0) 
    y = np.delete(ir3_y, (0), axis=0) 
    sub = np.delete(ir3_sub, (0), axis=0)
    sub = sub.astype(np.uint8) # convert from float to int

    xys_info = 'TWristAR e4 wristband structured 6-activity zip files\n'
    xys_info += '\n'.join([str(elem) for elem in zip_flist]) # conv list to string
    xys_info += '\nTime steps =' + str(time_steps) + ', Step =' + str(step) + ', no resample\n'
    xys_info += 'Final Shapes\n'
    xys_info += "X shape " + str(X.shape) + " dtype = " + str(X.dtype) + "\n"
    xys_info += "y shape " + str(y.shape) + " dtype = " + str(y.dtype) + "\n"
    xys_info += "sub shape " + str(sub.shape) + " dtype = " + str(sub.dtype) + "\n"
    xys_info += "Channel names:" + str(channel_list) + "\n"
    # Get final counts for label ndarray - not quite as easy as pandas df
    xys_info += "Final Label Counts\n"
    unique, counts = np.unique(y, return_counts=True)
    xys_info += str(np.asarray((unique, counts)).T)
    xys_info += "\nSamples per Subject\n"
    unique, counts = np.unique(sub, return_counts=True)
    xys_info += str(np.asarray((unique, counts)).T)
    return X, y, sub, xys_info
if interactive:
    X, y, sub, xys_info = get_ir3()
    headers = ("array","shape", "object type", "data type")
    mydata = [("X:", X.shape, type(X), X.dtype),
            ("y:", y.shape ,type(y), y.dtype),
            ("sub:", sub.shape, type(sub), sub.dtype)]
    print(tabulate(mydata, headers=headers))

Local TWristAR.zip found, skipping download
Found existing TWristAR directory, skipping unzip
Processing  ./TWristAR/sub1/1574621345_A01F11.zip
Unzipping ./TWristAR/sub1/1574621345_A01F11.zip in ./TWristAR_temp
./TWristAR_temp/ACC.csv Sample frequency =  32  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
./TWristAR_temp/BVP.csv Sample frequency =  64  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:11
./TWristAR_temp/EDA.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
./TWristAR_temp/TEMP.csv Sample frequency =  4  Hz
File start time =  Sun, 24 Nov 2019 18:49:05
File end time   =  Sun, 24 Nov 2019 18:58:10
IR1 full dataframe shape (17501, 7)
Tag info (button presses) from tags.csv
    UTC_time          Local Time
0 1574621375.17 Sun, 24 Nov 2019 18:49:35
1 1574621774.22 Sun, 24 Nov 2019 18:56:14
Label Counts - # samples before s

In [ ]:
if True: #change to true to save files interactively
    output_dir = '/content/drive/MyDrive/Processed_Datasets/UE4W/all_7_ch'
    if (os.path.isdir(output_dir)):
        #quick check for existing files, '.ipynb_checkpoints' file 
        #makes it more complicated to see if directory is empty
        if (not os.path.isfile(output_dir + '/X.npy')):
            readme = 'Unlabeled data from UE4W Repository, all 30 files\n'
            readme += 'this version for fusion learned reps paper.\n'
            readme += 'Lee Hinkle, IMICS lab, December 15, 2022\n'
            readme += ' Array    shape           data type\n'
            readme += '-------  ---------------  -----------\n'
            readme += 'X:       (331964, 96, 4)  float32\n'
            readme += 'y:       (331964, 1)      <U10\n'
            readme += 'sub:     (331964, 1)      uint8\n'
            readme += "Channels: ['accel_x', 'accel_y', 'accel_z', 'accel_ttl', 'bvp', 'eda', 'p_temp']\n"
            readme += 'first row [-8.906250e-01  2.031250e-01  4.531250e-01  1.970335e-02 -0.000000e+00 0.000000e+00  3.250000e+01]\n'
            readme += "Note: y is all 'unk' and sub is 1, so waste of space\n"

            with open(output_dir + '/'+"readme.txt", "w") as file_object:
                file_object.write(readme)

            if True:
                np.save(output_dir + '/'+'X.npy',X)
                np.save(output_dir + '/'+'y.npy',y)
                np.save(output_dir + '/'+'sub.npy',sub)
        else:
            print("Error "+output_dir+" contains X.npy, please delete files")
    else:
        print(output_dir + " not found, please create directory") 

In [ ]:
def limit_channel_ir3(ir3_X, 
                      all_channel_list = ['accel_x', 'accel_y', 'accel_z', 'accel_ttl', 'bvp', 'eda', 'p_temp'],
                      keep_channel_list = ["accel_ttl"]):
    """Pass the full ir3_X array with all channels, the stored all_channel_list
    that was extracted from the ir1 dataframe column names, and a 
    keep_channel_list.  Matching channels will be kept, all others dropped.
    This would have been much easier at IR1 but that would precluded channel 
    experiments and by channel feature representations.
    This is really new code, I'm leaving in some commented statements for now"""
    ch_idx = []
    # should add check here for channels not in list
    for i in keep_channel_list:
        ch_idx.append(all_channel_list.index(i)) 
    if verbose:
        print("Keeping X columns at index", ch_idx)
    new_X = ir3_X[:,:,ch_idx]
    return new_X
if interactive:
    print("all_channel_list", all_channel_list)
    print("starting X shape", my_X.shape)
    print("first row", my_X[0,0,:])
    my_new_X = limit_channel_ir3(my_X,
                                 keep_channel_list = ['accel_ttl','p_temp'])
    print("ending X shape", my_new_X.shape)
    print("first row", my_new_X[0,0,:])

In [ ]:
def twristar_load_dataset(
    verbose = False,
    use_saved_xysub = False, # get X,y,sub from zip, True = faster to used saved ones
    incl_val_group = False, # split train into train and validate
    split_subj = dict
                (train_subj = [1,2],
                validation_subj = [],
                test_subj = [3]),
    keep_channel_list = ['accel_ttl'],
    one_hot_encode = True, # make y into multi-column one-hot, one for each activity
    return_info_dict = False, # return dict of meta info along with ndarrays
    suppress_warn = False # special case for stratified warning
    ):
    """Downloads the TWristAR dataset from Zenodo, processes the data, and
    returns arrays by separating into _train, _validate, and _test arrays for
    X and y based on split_sub dictionary."""
    log_info = "Generated by TWristAR_load_data.ipynb\n"
    today = date.today()
    log_info += today.strftime("%B %d, %Y") + "\n"
    log_info += "sub dict = " + str(split_subj) + "\n"
    if (not use_saved_xysub):
        X, y, sub, xys_info = get_ir3()
    else:
        # read previously stored X, y, sub arrays instead of creating from zip
        # saves time when running multiple train/test split experiments
        # e.g. passing different subject dictionaries to this method
        # create & save X, y, sub using Leotta_2021_get_X_y_sub.ipynb interactively
        # this is not a big deal for TWristAR which is purposely small.
        input_dir = '/content/drive/MyDrive/Processed_Datasets/shl/ir3_20hz'
        X = np.load(input_dir + '/'+'X.npy')
        y = np.load(input_dir + '/'+'y.npy')
        sub = np.load(input_dir + '/'+'sub.npy')
        log_info += "X, y, sub loaded from "+str(input_dir)+"\n"
    # Drop unwanted channels from X
    log_info += "Keeping channels" + str(keep_channel_list) + "\n"
    X = limit_channel_ir3(X, keep_channel_list = keep_channel_list)
    # write initial array info to log_info
    headers = ("Initial Array","shape", "object type", "data type")
    mydata = [("X", X.shape, type(X), X.dtype),
              ("y", y.shape, type(y), y.dtype),
              ("sub", sub.shape, type(sub), sub.dtype)]
    if (verbose):
        print(tabulate(mydata, headers=headers))
    log_info += tabulate(mydata, headers=headers) + "\n"
    
 
    #One-Hot-Encode y...there must be a better way when starting with strings
    #https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
    # Need to look at https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

    if (one_hot_encode):
        # integer encode
        y_vector = np.ravel(y) #encoder won't take column vector
        le = LabelEncoder()
        integer_encoded = le.fit_transform(y_vector) #convert from string to int
        name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        if (verbose):
            print("One-hot-encoding: category names -> int -> one-hot \n")
            print(name_mapping) # seems risky as interim step before one-hot
        log_info += "One Hot:" + str(name_mapping) +"\n\n"
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        y=onehot_encoded.astype('uint8')
        #return X,y
    # split by subject number pass in dictionary
    sub_num = np.ravel(sub[ : , 0] ) # convert shape to (1047,)
    # this code is different from typical due to limited subjects,
    # all not test subjects data is placed into train which is then 
    # split using stratification - validation group is not sub independent
    train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj'] + 
                                        split_subj['validation_subj']))
    x_train = X[train_index]
    y_train = y[train_index]
    if (incl_val_group):
        if not suppress_warn:
            print("Warning: Due to limited subjects the validation group is a stratified")
            print("90/10 split of the training group.  It is not subject independent.")
        # split training into training + validate using stratify - note that the
        # validation set is not subject independent (hard to achieve with limited
        # subjects).   The test set however is subject independent and as a result
        # will have much lower accuracy.  Another option is to tag a few of the
        # activities for inclusion in validation.  See
        # https://github.com/imics-lab/Semi-Supervised-HAR-e4-Wristband
        # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
        x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.10, random_state=42, stratify=y_train)

    test_index = np.nonzero(np.isin(sub_num, split_subj['test_subj']))
    x_test = X[test_index]
    y_test = y[test_index]

    headers = ("Returned Array","shape", "object type", "data type")
    mydata = [("x_train:", x_train.shape, type(x_train), x_train.dtype),
                    ("y_train:", y_train.shape ,type(y_train), y_train.dtype)]
    if (incl_val_group):
        mydata += [("x_validation:", x_validation.shape, type(x_validation), x_validation.dtype),
                        ("y_validation:", y_validation.shape ,type(y_validation), y_validation.dtype)]
    mydata += [("x_test:", x_test.shape, type(x_test), x_test.dtype),
                    ("y_test:", y_test.shape ,type(y_test), y_test.dtype)]
    if (verbose):
        print(tabulate(mydata, headers=headers))
    log_info += tabulate(mydata, headers=headers)
    if (incl_val_group):
        if (return_info_dict):
            return x_train, y_train, x_validation, y_validation, x_test, y_test, log_info
        else:
            return x_train, y_train, x_validation, y_validation, x_test, y_test
    else:
        if (return_info_dict):
            return x_train, y_train, x_test, y_test, log_info
        else:
            return x_train, y_train, x_test, y_test


# Main is setup to be a demo and bit of unit test.

In [ ]:
if __name__ == "__main__":
    print("Get TWristAR using defaults - simple and easy!")
    x_train, y_train, x_test, y_test \
                             = twristar_load_dataset()
    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print ('\n','-'*72)

    print("Get TWristAR with validation group, info file, and four channels\n")
    x_train, y_train, x_valid, y_valid, x_test, y_test, log_info \
                             = twristar_load_dataset(
                                 incl_val_group = True,
                                 keep_channel_list = ['accel_ttl','bvp',
                                                      'eda', 'p_temp'],
                                 return_info_dict = True)

    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_valid:", x_valid.shape, x_valid.dtype),
            ("y_valid:", y_valid.shape, y_valid.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print("\n----------- Contents of returned log_info ---------------")
    print(log_info)
    print("\n------------- End of returned log_info -----------------")
    print("Get TWristAR with validation group, no warn, and bvp only\n")
    x_train, y_train, x_valid, y_valid, x_test, y_test \
                             = twristar_load_dataset(
                                 incl_val_group = True,
                                 keep_channel_list = ['bvp'],
                                 return_info_dict = False,
                                 suppress_warn = True)
    print("This is a no output config - silent execution")
    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_valid:", x_valid.shape, x_valid.dtype),
            ("y_valid:", y_valid.shape, y_valid.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print("Get TWristAR with validation group, and accel only\n")
    x_train, y_train, x_valid, y_valid, x_test, y_test, log_accelxyz\
                             = twristar_load_dataset(
                                 incl_val_group = True,
                                 keep_channel_list = ['accel_x', 'accel_y', 'accel_z', 'accel_ttl'],
                                 return_info_dict = True,
                                 suppress_warn = True)
    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_valid:", x_valid.shape, x_valid.dtype),
            ("y_valid:", y_valid.shape, y_valid.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print("\n----------- Contents of returned log_info ---------------")
    print(log_accelxyz)
    print("\n------------- End of returned log_info -----------------")

Get TWristAR using defaults - simple and easy!

 Array     shape          data type
--------  -------------  -----------
x_train:  (2077, 96, 1)  float32
y_train:  (2077, 6)      uint8
x_test:   (1091, 96, 1)  float32
y_test:   (1091, 6)      uint8

 ------------------------------------------------------------------------
Get TWristAR with validation group, info file, and four channels

90/10 split of the training group.  It is not subject independent.

 Array     shape          data type
--------  -------------  -----------
x_train:  (1869, 96, 4)  float32
y_train:  (1869, 6)      uint8
x_valid:  (208, 96, 4)   float32
y_valid:  (208, 6)       uint8
x_test:   (1091, 96, 4)  float32
y_test:   (1091, 6)      uint8

----------- Contents of returned log_info ---------------
Generated by TWristAR_load_data.ipynb
December 13, 2022
sub dict = {'train_subj': [1, 2], 'validation_subj': [], 'test_subj': [3]}
Keeping channels['accel_ttl', 'bvp', 'eda', 'p_temp']
Initial Array    shape          o

#Save arrays to drive
This is common code and untested - TWristAR is small so download and processing is fast.

For some of the larger datsets it is a big time benefit to store the arrays either before or after train/test split.  

In [ ]:
if False: #change to true to save files interactively
    output_dir = '/content/drive/MyDrive/Processed_Datasets/TWristAR/all-sensors'
    if (os.path.isdir(output_dir)):
        #quick check for existing files, '.ipynb_checkpoints' file 
        #makes it more complicated to see if directory is empty
        if (not os.path.isfile(output_dir + '/X.npy')):
            summary = "TWristAR data\n"
            summary += "Saved to " + output_dir + "\n"
            summary += "Generated by TWristAR_load_data.ipynb"
            summary += " on " + time.strftime('%b-%d-%Y_%H%M', time.localtime())
            summary += "this version for fusion of learned representation work\n"
            summary += "contains data from all 4 e4 sensors"
            info_fname = output_dir +'/'+'README.txt'
            full_info = summary + "\n" + xys_info + "\n"
            print(full_info)

            with open(info_fname, "w") as file_object:
                file_object.write(full_info)

            if True:
                np.save(output_dir + '/'+'X.npy',X)
                np.save(output_dir + '/'+'y.npy',y)
                np.save(output_dir + '/'+'sub.npy',sub)
        else:
            print("Error "+output_dir+" contains X.npy, please delete files")
    else:
        print(output_dir + " not found, please create directory") 

In [ ]:
if interactive:
    import matplotlib.pyplot as plt # for plotting

In [ ]:
# Plot y - must convert to numeric first
def plot_activities():
    uniques, y_num = np.unique(y, return_inverse=True)
    print (uniques)
    plt.plot(y_num) 
    plt.show()
if (interactive):
    plot_activities()

In [ ]:
def plot_subjects():
    uniques, s_num = np.unique(sub, return_inverse=True)
    print (uniques)
    plt.plot(s_num) 
    plt.show()
if (interactive):
    plot_subjects()